In [5]:
%run ../../utils/commonImports.py
%run ../../utils/tradingImports.py
from exputils import *

%matplotlib inline

In [15]:
from trading.dataManager import DataManager

class OhlcDataManager(DataManager):
    # expected prices to be pandas dataframe with columns: timestamp, open, close, high, low, volume
    def __init__(self, prices, data):
        price_cols = ['open', 'low', 'high', 'close']
        self.prices = prices[price_cols].values
        self.maxTime = self.prices.shape[0]
        self.data = data.values
        self.time = 0
        self.inner_time = 0

    def tick(self, history_len):
        if not self.has_tick():
            return None, None
        time_i = int(self.time)
        price = self.prices[time_i, self.inner_time]
        first = time_i - history_len if time_i - history_len > 0 else 0
        history = self.data[first:time_i, :]
        self.inner_time += 1
        if self.inner_time == 4:
            self.inner_time = 0
            self.time = round(self.time + 0.6, 1)

        self.time = round(self.time + 0.1, 1)
        return history, price

    def has_tick(self):
        return self.time < self.maxTime

In [16]:
dropbox_dir = '/Users/david.vesely/Dropbox/My work/krypl-project'
features = ['volume']
data = read_test('BTC_LTC', dropbox_dir)

In [17]:
%%timeit
dm = OhlcDataManager(data[['open', 'low', 'high', 'close']], data[features])
while dm.has_tick():
    dm.tick(2)

173 ms ± 451 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%prun
dm = OhlcDataManager(data[['open', 'low', 'high', 'close']], data[features])
while dm.has_tick():
    dm.tick(2)

         375402 function calls (375334 primitive calls) in 0.253 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    87360    0.121    0.000    0.189    0.000 <ipython-input-11-1a130491655f>:13(tick)
   109200    0.055    0.000    0.055    0.000 {built-in method builtins.round}
        1    0.045    0.045    0.253    0.253 <string>:2(<module>)
   174721    0.027    0.000    0.027    0.000 <ipython-input-11-1a130491655f>:28(has_tick)
        3    0.000    0.000    0.000    0.000 {built-in method posix.stat}
        3    0.000    0.000    0.000    0.000 {pandas._libs.algos.take_2d_axis0_float64_float64}
      629    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
      389    0.000    0.000    0.000    0.000 {built-in method builtins.getattr}
       61    0.000    0.000    0.000    0.000 {built-in method numpy.array}
        6    0.000    0.000    0.001    0.000 base.py:253(__new__)
       12    0.000  